In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from matplotlib import pyplot as plt

import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
type(trainset.targets)
type(trainset.data)

numpy.ndarray

In [ ]:
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
# testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

foreground_classes = {'plane', 'car', 'bird'}

background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

fg1,fg2,fg3 = 0,1,2

In [ ]:
is_fg = [np.where(np.array(trainset.targets)==fg1)[0] , np.where(np.array(trainset.targets)==fg2)[0], np.where(np.array(trainset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   29    30    35 ... 49987 49991 49995] (15000,)


In [ ]:
all_index = np.arange(0,50000)
all_index

array([    0,     1,     2, ..., 49997, 49998, 49999])

In [ ]:
train_labels = np.array(trainset.targets)
train_labels.shape

(50000,)

In [ ]:
train_labels[is_fg]=1
train_labels[list(set(all_index)-set(is_fg))]=0
train_labels

array([0, 0, 0, ..., 0, 1, 1])

In [ ]:
np.count_nonzero(train_labels)

15000

In [ ]:
trainset.targets = train_labels

In [ ]:
is_fg = [np.where(np.array(testset.targets)==fg1)[0] , np.where(np.array(testset.targets)==fg2)[0], np.where(np.array(testset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   3   10   21 ... 9970 9982 9989] (3000,)


In [ ]:
all_index = np.arange(0,10000)
all_index

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [ ]:
test_labels = np.array(testset.targets)
test_labels.shape

(10000,)

In [ ]:
test_labels[is_fg]=1
test_labels[list(set(all_index)-set(is_fg))]=0
test_labels

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
np.count_nonzero(test_labels)

3000

In [ ]:
testset.targets = test_labels

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=False)

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=0)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=0)
    self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=0)
    self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.batch_norm1 = nn.BatchNorm2d(32)
    self.batch_norm2 = nn.BatchNorm2d(128)
    self.dropout1 = nn.Dropout2d(p=0.05)
    self.dropout2 = nn.Dropout2d(p=0.1)
    self.fc1 = nn.Linear(128,64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 10)
    self.fc4 = nn.Linear(10, 2)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(self.batch_norm1(x))

    x = (F.relu(self.conv2(x)))
    x = self.pool(x)
    
    x = self.conv3(x)
    x = F.relu(self.batch_norm2(x))

    x = (F.relu(self.conv4(x)))
    x = self.pool(x)
    x = self.dropout1(x)

    x = self.conv5(x)
    x = F.relu(self.batch_norm2(x))

    x = (F.relu(self.conv6(x)))
    x = self.pool(x)

    x = x.view(x.size(0), -1)

    x = self.dropout2(x)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.dropout2(x)
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

In [ ]:
where_net = CNN()#.double()
where_net = where_net.to("cuda")

In [ ]:
where_net

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batch_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0.05, inplace=False)
  (dropout2): Dropout2d(p=0.1, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=

In [ ]:
import torch.optim as optim
criterion_where = nn.CrossEntropyLoss()
optimizer_where = optim.SGD(where_net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
acti = []
loss_curi = []
epochs = 1000
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer_where.zero_grad()

        # forward + backward + optimize
        outputs = where_net(inputs)
        loss = criterion_where(outputs, labels)
        loss.backward()
        optimizer_where.step()

        # print statistics
        running_loss += loss.item()
        mini_batch = 50
        if i % mini_batch == mini_batch-1:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / mini_batch))
            ep_lossi.append(running_loss/mini_batch) # loss per minibatch
            running_loss = 0.0
            
    if(np.mean(ep_lossi) <= 0.003):
      break;
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch            

print('Finished Training')

[1,    50] loss: 0.621
[1,   100] loss: 0.606
[1,   150] loss: 0.601
[2,    50] loss: 0.495
[2,   100] loss: 0.458
[2,   150] loss: 0.441
[3,    50] loss: 0.387
[3,   100] loss: 0.384
[3,   150] loss: 0.383
[4,    50] loss: 0.326
[4,   100] loss: 0.329
[4,   150] loss: 0.328
[5,    50] loss: 0.292
[5,   100] loss: 0.291
[5,   150] loss: 0.292
[6,    50] loss: 0.257
[6,   100] loss: 0.260
[6,   150] loss: 0.257
[7,    50] loss: 0.221
[7,   100] loss: 0.231
[7,   150] loss: 0.231
[8,    50] loss: 0.190
[8,   100] loss: 0.196
[8,   150] loss: 0.200
[9,    50] loss: 0.165
[9,   100] loss: 0.183
[9,   150] loss: 0.184
[10,    50] loss: 0.139
[10,   100] loss: 0.154
[10,   150] loss: 0.161
[11,    50] loss: 0.115
[11,   100] loss: 0.125
[11,   150] loss: 0.129
[12,    50] loss: 0.109
[12,   100] loss: 0.135
[12,   150] loss: 0.123
[13,    50] loss: 0.094
[13,   100] loss: 0.112
[13,   150] loss: 0.101
[14,    50] loss: 0.081
[14,   100] loss: 0.092
[14,   150] loss: 0.093
[15,    50] loss: 0

In [ ]:
torch.save(where_net.state_dict(),"/content/drive/My Drive/Research/Cheating_data/Focus_net_weights/focus_net_6layer_cnn"+".pt")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = where_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 50000 train images: 99 %
50000 49982


In [ ]:
correct = 0
total = 0
out = []
pred = []
where_net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= where_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 89 %
10000 8978
